In [10]:
import time
import numpy as np
from IPython.display import clear_output

R = 5
C = 5
A = 5

N = R * C

epsilon = 0.1
# epsilon = 0.0

action_grid = [
[1, 1, 1, 1, 1],
[1, 1, 1, 1, 1],
[1, 1, 1, 1, 1],
[1, 1, 1, 1, 1],
[1, 1, 1, 1, 1],
]
# action_grid = (np.random.rand(R, C) * A).astype(int)

# Policy Matrix.
# pi[i][j] mean the probability of `at state i choose action j`.
pi = np.zeros((N, A))
for r_ in range(R):
    for c_ in range(C):
        pi[r_ * C + c_] = 1 / A

# Block states.
blocks = np.array([
[-1,  -1,  -1,  -1, -1],
[-1, -10, -10,  -1, -1],
[-1,  -1, -10,  -1, -1],
[-1, -10,   0, -10, -1],
[-1, -10,  -1,  -1, -1],
])
# blocks = np.array([
# [0,   0,   0,   0,  0],
# [0, -10, -10,   0,  0],
# [0,   0, -10,   0,  0],
# [0, -10,   1, -10,  0],
# [0, -10,   0,   0,  0],
# ])

action_to_direction = {}
action_to_direction[0] = np.array([-1, 0])
action_to_direction[1] = np.array([0, 1])
action_to_direction[2] = np.array([1, 0])
action_to_direction[3] = np.array([0, -1])
action_to_direction[4] = np.array([0, 0])

gamma = 0.9

# State Values.
v = np.ones((R * C, 1))

# Episode Length.
# If not decrease epsilon, this E should be large for convergence.
E = 100

q_table = np.zeros((N, A))

num_episode = 300

alpha = 0.1
r_start = 0
c_start = 0
r_target = 3
c_target = 2

for episode in range(num_episode):
    # 1. Random start.
    state = (r_start, c_start)
    a_start = np.random.choice(range(A), size=1, replace=False, p=pi[r_start * C + c_start])[0]

    # 2. Generate a Episode.
    episode_history = []
    for e in range(E):
        r_, c_ = state
        a = a_start
        if e != 0:
            a = np.random.choice(range(A), size=1, replace=False, p=pi[r_ * C + c_])[0]

        delta = action_to_direction[a]
        r_p = r_ + delta[0]
        c_p = c_ + delta[1]

        if r_p >= R or r_p < 0 or c_p >= C or c_p < 0:
            reward = -10
            next_state = (r_, c_)
        else:
            reward = blocks[r_p][c_p]
            next_state = (r_p, c_p)

        episode_history.append((state, a, reward))
        state = next_state

        if next_state[0] == r_target and next_state[1] == c_target:
            break
    
    for e in range(len(episode_history) - 1):
        (r_, c_), a, reward = episode_history[e]
        (r_next, c_next), a_next, _ = episode_history[e + 1]
        # Update q-value.
        TD_target = reward + gamma * q_table[r_next * C + c_next][a_next]
        TD_error = q_table[r_ * C + c_][a] - TD_target
        q_table[r_ * C + c_][a] -= alpha * TD_error
        # Update Policy.
        max_a = np.argmax(q_table[r_ * C + c_])
        pi[r_ * C + c_] = epsilon / A
        pi[r_ * C + c_][max_a] = 1 - epsilon / A * (A - 1)

    ## Enhance Exploitation.
    if epsilon > 0.001:
        epsilon -= 0.001

    time.sleep(0.2)
    clear_output(wait=True)

    print('len(episode_history):', len(episode_history))
    action_grid = np.zeros((R, C))
    for r_ in range(R):
        for c_ in range(C):
            a = np.argmax(pi[r_ * C + c_])
            action_grid[r_][c_] = a
    print(f'{episode}th Action Grid:\n', action_grid)

clear_output(wait=True)

action_grid = np.zeros((R, C))
for r_ in range(R):
    for c_ in range(C):
        a = np.argmax(pi[r_ * C + c_])
        action_grid[r_][c_] = a
print(f'Converged Action Grid:\n', action_grid, flush=True)




Converged Action Grid:
 [[1. 1. 1. 1. 2.]
 [2. 2. 2. 2. 2.]
 [2. 1. 2. 3. 2.]
 [1. 1. 0. 3. 2.]
 [1. 1. 0. 3. 3.]]
